<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Plugins for Decision Recommendation

This module will show how to perform a more advanced search using the Bing Search SDK. Any .NET SDK that is used for decision making, business processes, interacts with data etc. can be integrated into AI Orchestration with Semantic Kernel. 

This plugin will be used to recommend whether to make a decision if a baseball player should be inducted into the Hall of Fame.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [2]:
// Create a semantic kernel with Azure OpenAI chat completion
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

### Step 2 - Create a custom WebSearch Results Plugin

Execute the next two cells to:
* Create a custom WebSearchResultsPlugin
* The plugin includes a native C# function called GetWebSearchResults
* GetWebSearchResults uses the Bing Search SDK to perform an advanced web search
* Notice that the GetWebSearchResults method is decorated with the KernelFunction attribute. This lets Semantic Kernel know that this is a kernel function that can be imported  
* Both the GetWebSearchResults method and the nameOfBaseballPLayer parameter have descriptions  

In [3]:
public class WebSearchResult
{
    public int Id { get; set; } = 0;
    public string Name { get; set; } = string.Empty;
    public string Snippet { get; set; } = string.Empty;
    public string Url { get; set; } = string.Empty;
}

public class WebSearchResultsPlugin
{
    [KernelFunction]
    [Description("Retrieves the web search results with identified URL sources for a given baseball player.")]
    public async Task<string> GetWebSearchResults(
        [Description("Name of the baseball player to search.")]
        string nameOfBaseballPlayer)
    {
        var configurationBuilder = new ConfigurationBuilder()
            .SetBasePath(Directory.GetCurrentDirectory())
            .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
            .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
        var config = configurationBuilder.Build();
        var bingSearchAPIKey = config["BingSearch:APIKey"];

        var webSearchResultsString = "Web search results:\r\n\r\n";
        var footNotes = string.Empty;
        var bingSearchId = 0;
        var webSearchResults = new List<WebSearchResult>();

        var bingSearchClient = new WebSearchClient(new ApiKeyServiceClientCredentials(bingSearchAPIKey));
        var bingWebData = await bingSearchClient.Web.SearchAsync(query: "baseball hall of fame " + nameOfBaseballPlayer, count: 8);

        if (bingWebData?.WebPages?.Value?.Count > 0)
        {
            // Itertate over the Bing Web Pages (Non-Cache Results)
            foreach (var bingWebPage in bingWebData.WebPages.Value)
            {
                bingSearchId++;

                webSearchResultsString += string.Format("[{0}]: \"{1}: {2}\"\r\nURL: {3}\r\n\r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Snippet, bingWebPage.Url);

                footNotes += string.Format("[{0}]: {1}: {2}  \r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Url);

                webSearchResults.Add(new WebSearchResult
                {
                    Id = bingSearchId,
                    Name = bingWebPage.Name,
                    Snippet = bingWebPage.Snippet,
                    Url = bingWebPage.Url
                });
            }
        }

        return webSearchResultsString;
    }
}
semanticKernel.ImportPluginFromType<WebSearchResultsPlugin>();

### Step 3 - Recommend a Decision using the internet Knowledge Graph

The custom plugin above performs an advanced Bing Search query, maintains the sources (sites) that were used for search and keeps the index of each site used. This is important as it not only provides knowledge grounding information but it also provides a framework to create citations as the LLM reasons over the decision recommendation. This allows AI architects to build "AI explainability" into their AI orchestration pipelines.

Execute the cell below to understand the LLMs reasoning process, it's final decision recommendation on Mike Trout's hall of fame induction and finally the sources that were used to arrive at that conclusion.

In [4]:
// Configure the OpenAI prompt execution settings with Auto Invoking of Kernel Functions
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let's invoke the prompt and stream the results
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the information available, I would recommend that Mike Trout should be inducted into the Hall of Fame. Here's why:

1. **Eligibility and Qualifications**: Mike Trout has satisfied the Hall of Fame election eligibility rule by playing in ten Major League championship seasons [1]. This is a fundamental criterion for consideration, and Trout meets this requirement.

2. **Career Achievements**: Trout is an 11-time MLB All-Star and a three-time American League (AL) Most Valuable Player (MVP), having won the award in 2014, 2016, and 2019. He also finished second in the MVP voting in 2012, 2013, and 2015 [4]. These accolades highlight his consistent performance and dominance in the league.

3. **Statistical Performance**: Despite injuries impacting his career [3][5], Trout's performance metrics remain impressive. His ability to maintain high standards in batting and fielding, even with fewer games, underscores his skill and resilience.

4. **Comparative Analysis**: Articles and analy

### Step 4 - Understanding Auto Function Calling Behavior

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [5]:
// Method for printing the functions metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: WebSearchResultsPlugin
  Function Name: GetWebSearchResults
  Function Description: Retrieves the web search results with identified URL sources for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: Name of the baseball player to search.
        default: ''

